In [ ]:
import pandas as pd
from sklearn.metrics import r2_score,make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from utils.metrics import partsMetrics,allMetrics
from utils.plots import exp_vs_predict,variation_with_co2,variation_with_wb,variation_with_FA,variation_with_RH,variation_with_T

In [ ]:
df=pd.read_csv('processed_outliers.csv')
print('Data shape:',df.shape)

In [ ]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values
print(X.shape)
print(y.shape)

In [ ]:
param_grid={
    'criterion': ['squared_error','absolute_error','friedman_mse','poisson'],
    'n_estimators': [10,50,100,200,300],
    'max_features': [None,1,2,3,4,5,'sqrt','log2'],
    'max_depth': [None,2,4,5,6],
    'min_samples_leaf': [1,3,4,5],
    'min_samples_split': [2,4,6,8,10],
    'bootstrap': [True,False],
    'min_weight_fraction_leaf': [0.0,.1]
}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

def train_model(X_train,y_train):
    reg = RandomForestRegressor(random_state=0,n_jobs=-1)
    grid=GridSearchCV(reg,param_grid,cv=5,scoring=make_scorer(r2_score),n_jobs=2)
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid

In [ ]:
model=train_model(X_train,y_train)

In [ ]:
best_params={'bootstrap': False, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 300}
reg=RandomForestRegressor(**best_params,random_state=0)
partsMetrics(df,reg)

In [ ]:
allMetrics(df,reg)

In [ ]:
exp_vs_predict(df,reg,'RF model')

In [ ]:
variation_with_co2(df,reg)

In [ ]:
variation_with_wb(df,reg)

In [ ]:
variation_with_FA(df,reg)

In [ ]:
variation_with_RH(df,reg)

In [ ]:
variation_with_T(df,reg)